### **Cabecera**

Gutierrez Viveros Cristian Rogelio 

5BV1 

Ingenieria en Inteligencia Artificial 

Ultima modificacion: 12/01/2024

Práctica 5. Analisis de sentimientos

A lo largo de esta practica, utilizaremos diversos modelos para el analisis de sentimientos, utilizando modelos de machine learning, diccionarios y redes neuronales. Ademas de que realizaremos un analisis de una coleccion de reseñas de comida fina en amazon y finalmente compararemos los distintos modelos al ponerlos a prueba con el dataset normalizado y vectorizado segun el modelo que corresponda

## **Modulos**

In [4]:
import os
import csv
import random
import re
from sklearn.svm import SVC
import spacy
import en_core_web_sm
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
import pysentiment2 as ps
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
import nltk
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\Crist\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Crist\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Crist\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from numpy import asarray
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from sklearn.preprocessing import LabelEncoder
from numpy import zeros
from keras.preprocessing.text import one_hot


## **Funciones**

In [6]:
# Leemos el CSV
def leer_datos(path):
    data = []
    with open(path, 'r', encoding='utf-8') as archivo: 
        reader = csv.reader(archivo)
        header = next(reader, None)  # Obtener la primera línea como encabezado
        for linea in reader:
            data.append(linea)

    return header, data

def leer_datos_noheader(path):
    data = []
    with open(path, 'r', encoding='utf-8') as archivo: 
        reader = csv.reader(archivo)
        for linea in reader:
            data.append(linea)
    return data

def randomize(lista):
    randomized = lista[:]  
    random.shuffle(randomized)
    return randomized

# Elimina etiquetas HTML y comentarios dentro del propio texto como indicaciones extra entre parentesis o comentarios entre --
def eliminar_etiquetas(text):
    pattern = re.compile(r'<[^>]+>|\([^)]+\)|--[^\-]+--')
    doc = pattern.sub('', text)
    return doc

# Removemos apostrofes para quedarnos con palabras como cant couldnt o dont
def remover_apostrofes(text):
    pattern = r"\\?'"
    doc = re.sub(pattern, '', text)
    return doc
# Nos quedamos con solo alfabeto eliminando caracteres especiales    
def remover_especiales(text):
    pattern = r"[^a-z\s]"
    doc = re.sub(pattern, ' ', text)
    return doc
# Quitamos dobles espacios para tener texto espaciado solamente por un espacio por palabra
def dobles_espacios(text):
    pattern = r'\s+'
    doc = re.sub(pattern, ' ', text)
    return doc
    
# Removemos stop words unicamente en ingles
def remove_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# Lematizacion mediante spacy
nlp = spacy.load("en_core_web_sm")
def lemmatize(doc):
    docs = list(nlp.pipe([doc]))
    lemas = ' '.join([token.lemma_ for doc in docs for token in doc])
    return lemas




Funciones destinadas para analisis de sentimientos con diccionarios

In [7]:
def hiv4_polarity(text,hiv4):
    tokens = hiv4.tokenize(text)  
    score = hiv4.get_score(tokens)

    polarity = score['Polarity']
    if polarity > 0.33:
        return "Positivo"
    elif polarity < -0.33:
        return "Negativo"
    else:
        return "Neutral"

def analyze_sentiment(text):
    # Tokenizar el texto
    tokens = word_tokenize(text.lower())

    # Obtener palabras positivas y negativas del Opinion Lexicon
    pos_words = set(opinion_lexicon.positive())
    neg_words = set(opinion_lexicon.negative())

    # Calcular el sentimiento
    pos_score = sum(word in pos_words for word in tokens)
    neg_score = sum(word in neg_words for word in tokens)

    # Determinar la orientación general
    if pos_score > neg_score:
        return "Positivo"
    elif pos_score < neg_score:
        return "Negativo"
    else:
        return "Neutro"


Funciones de vectorizacion de textos

In [2]:
def vectorizar_tfidf(xd):
    # TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(xd)
    
    # palabras únicas
    feature_names_tfidf = vectorizer.get_feature_names_out()

    print("Tamaño de Palabras únicas:", len(feature_names_tfidf))
    return tfidf_matrix

def vectorizar_onehot(xd):
    # One-Hot Encoding
    vectorizer = CountVectorizer(binary=True)
    onehot_matrix = vectorizer.fit_transform(xd)
    return onehot_matrix

## **Análisis exploratorio de datos y preprocesamiento**

Para la lectura de archivos, leeremos el CSV de las reseñas, tomando la primer linea como la cabecera con los titulos de cada columna, y el resto en data

In [54]:
directory = os.getcwd()
path = (directory+"\Reviews.csv")
header, data = leer_datos(path)

In [21]:
print("hay",len(data),"datos. Cada uno con",len(header),"caracteristicas")
print("columnas:")
print(header,"\n")
print(data[1], "\n")
print(data[2])

hay 568454 datos. Cada uno con 10 caracteristicas
columnas:
['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'] 

['2', 'B00813GRG4', 'A1D87F6ZCVE5NK', 'dll pa', '0', '0', '1', '1346976000', 'Not as Advertised', 'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".'] 

['3', 'B000LQOCH0', 'ABXLMWJIXXAIN', 'Natalia Corres "Natalia Corres"', '1', '1', '4', '1219017600', '"Delight" says it all', 'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion,

In [52]:

print("Datos de la primera fila:")
for i in range(len(header)):
    print(header[i] + ":", data[1][i], " (tipo:", type(data[1][i]), ")")

Datos de la primera fila:
Id: 2  (tipo: <class 'str'> )
ProductId: B00813GRG4  (tipo: <class 'str'> )
UserId: A1D87F6ZCVE5NK  (tipo: <class 'str'> )
ProfileName: dll pa  (tipo: <class 'str'> )
HelpfulnessNumerator: 0  (tipo: <class 'str'> )
HelpfulnessDenominator: 0  (tipo: <class 'str'> )
Score: 1  (tipo: <class 'str'> )
Time: 1346976000  (tipo: <class 'str'> )
Summary: Not as Advertised  (tipo: <class 'str'> )
Text: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".  (tipo: <class 'str'> )


### Segmentacion

Guardamos el vector de reviews solamente con las columnas del score, el summary y el texto de la reseña

In [55]:
columnas = [6,8,9]
reviews = [[row[i] for i in columnas] for row in data]
print(reviews[2])

['4', '"Delight" says it all', 'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.']


Convertimos el score de las reseñas a Negativo, Neutral y Positivo

In [23]:
label_counter = [0,0,0]
for i in range(len(reviews)):
    if reviews[i][0] == '1' or reviews[i][0] == '2':
        reviews[i][0] = 'Negativo'
        label_counter[0] += 1
    elif reviews[i][0] == '3':
        reviews[i][0] = 'Neutral'
        label_counter[1] += 1
    else:
        reviews[i][0] = 'Positivo'
        label_counter[2] += 1
print(label_counter)

[82037, 42640, 443777]


Mezclamos las reseñas con su respectiva etiqueta y nos quedamos solamente con las primeras 40000 reseñas de cada etiqueta

In [24]:
reviews = randomize(reviews)
print(reviews[0])

['Positivo', 'Good', 'This is a tasty trail mix with a blend of nuts, sesame sticks, and chocolate covered soy pieces. Each packet is a single serving of 1.5 ounces, which is not very much - maybe about 2 tablespoons.  This was a good deal when I bought it for $20.77.  It is not a good deal at $43.77!']


In [25]:
reviews_compressed = []
label_check = [0,0,0]
for i in range(len(reviews)):
    if label_check[0] == label_check[1] == label_check[2] == 40000:
        break
    if reviews[i][0] == 'Positivo' and label_check[0] < 40000:
        reviews_compressed.append(reviews[i])
        label_check[0] += 1
    elif reviews[i][0] == 'Neutral' and label_check[1] < 40000:
        reviews_compressed.append(reviews[i])
        label_check[1] += 1
    elif reviews[i][0] == 'Negativo' and label_check[2] < 40000:
        reviews_compressed.append(reviews[i])
        label_check[2] += 1
print(label_check)
print(len(reviews_compressed))
reviews = reviews_compressed

[40000, 40000, 40000]
120000


## **Limpieza de Texto**

### Analisis

In [59]:
for i in range (20):
    print(reviews[i+555][2],"\n")

Don't waste your money on any of the Kettle brand potato chips.  I bought a case of these, and a case of the cheddar and sour cream.  Both cases ended up in the garbage can. 

DEfintely not as tasty as the Madhouse Munchies which are my family's favorite. Kettle's are dark/burnt,more broken bits, taste greasy-oily and not the light crunch. Oh well. 

I love sour food but this one I can't bear.  Too strong sour taste... and even when you open the bag, the sour smell is too strong.  I don't like it. 

Unless you really really really like vinegar - AVOID! Those chips should have been called "Vinegar and Sea Salt" - not "Sea Salt and Vinegar". 

I have loved Kettle Brand Sea Salt and Vinegar chips since the first time i tried them.  The fact that i was able to find them on-line for such a great price, was wonderful.  I would definitely make the purchase again.  THey are a quick and simple snack for lunch and it goes great with my cold sandwich. 

Used to eat the Spicy Thai flavor all the t

### Tecnicas de limpieza

In [27]:
for i in range(len(reviews)):
    reviews[i][2] = eliminar_etiquetas(reviews[i][2])

In [28]:
for i in range(len(reviews)):
    reviews[i][1] = reviews[i][1].lower()
    reviews[i][2] = reviews[i][2].lower()
    reviews[i][1] = remover_apostrofes(reviews[i][1])
    reviews[i][2] = remover_apostrofes(reviews[i][2])
    reviews[i][1] = remover_especiales(reviews[i][1])
    reviews[i][2] = remover_especiales(reviews[i][2])
    reviews[i][1] = dobles_espacios(reviews[i][1])
    reviews[i][2] = dobles_espacios(reviews[i][2])

In [29]:
reviews_fixed = reviews.copy()

In [30]:
for i in range(len(reviews)):
    if i % 10000 == 0:
        print("Bloque siendo leido (13 bloques en total)")
    reviews[i][1] = remove_stopwords(reviews[i][1])
    reviews[i][2] = remove_stopwords(reviews[i][2])
    reviews[i][1] = lemmatize(reviews[i][1])
    reviews[i][2] = lemmatize(reviews[i][2])

Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)
Bloque siendo leido (13 bloques en total)


### Guardado de resultados tras limpieza

In [31]:
reviews_save = reviews.copy()

Ya habia guardado las reseñas normalizadas con anterioridad, solo que lo corri posteriormente al comentar :)

In [6]:
filename = "Reviews_140k.csv"

with open(filename, mode='w', newline='', encoding='utf-8') as archivo_csv:
    writing = csv.writer(archivo_csv)

    for fila in reviews_save:
        writing.writerow(fila)

print(f"Se ha guardado como {filename} correctamente.")

NameError: name 'reviews_save' is not defined

In [40]:
for i in range (20):
    print(reviews[i],"\n")

['Positivo', 'good', 'tasty trail mix blend nuts sesame stick chocolate cover soy piece packet single serve ounce much maybe tablespoon good deal buy good deal'] 

['Positivo', 'good affordable', 'turn cut sugar diet purely health maintenance measure blood pressure bmi cholesterol blood sugar continue healthy age height despite inactive something get change want stay healthy lifestyle diet preferable manage use splenda packet per day beverage try sweetener least week i d like support stevia use could handle leafy taste even ice tea already leafy concern may much since many web sight warn aluminum buildup liver contact cargill try find amount splenda human trail involve would tell say consumption limit that s take star review wish splendas maker would forthcoming consumer contact legitimate concern tolerate sugar make gallon beverage add one tablespoon sugar per grams splenda soften metallic taste sweetener'] 

['Positivo', 'one favorite', 'love flavor like try different coffee flavor r

## **Transformación de datos**

Con el fin de no cargar una y otra vez la normalizacion, guarde los resultados de la normalizacion en un csv, aqui es donde lo abrimos para su transformacion 

In [12]:
directory = os.getcwd()
path = (directory+"\Reviews_140k.csv")
data_cleaned = leer_datos_noheader(path)

In [13]:
reviews_compressed = []
label_check = [0,0,0]
for i in range(len(data_cleaned)):
    if label_check[0] == label_check[1] == label_check[2] == 40000:
        break
    if data_cleaned[i][0] == 'Positivo' and label_check[0] < 40000:
        reviews_compressed.append(data_cleaned[i])
        label_check[0] += 1
    elif data_cleaned[i][0] == 'Neutral' and label_check[1] < 40000:
        reviews_compressed.append(data_cleaned[i])
        label_check[1] += 1
    elif data_cleaned[i][0] == 'Negativo' and label_check[2] < 40000:
        reviews_compressed.append(data_cleaned[i])
        label_check[2] += 1
print(label_check)
print(len(reviews_compressed))
data_cleaned = reviews_compressed

[40000, 40000, 40000]
120000


In [17]:
text = []
for i in range(len(data_cleaned)):
    text.append(data_cleaned[i][2])

In [62]:

print(len(text))
print(text[5])

120000
treat perfect disguise hide daily med little furball plus since cost bag store price low shipping fee bargain thank bunch


Realizamos la vectorizacion de los datos tomando la lista de los textos de las reseñas, en el caso de tfidf, ademas printeo el tamaño de palabras unicas

In [35]:
text_tfidf = vectorizar_tfidf(text)

Tamaño de Palabras únicas: 52666


In [64]:
text_onehot = vectorizar_onehot(text)

Imprimimos una muestra de como se observa TF-IDF y one-hot encoding

In [36]:
print(text_tfidf[0])

  (0, 6283)	0.10205401103059528
  (0, 11610)	0.3938934447094808
  (0, 19528)	0.1808582127790854
  (0, 45350)	0.2651735766030554
  (0, 28161)	0.1740201202973761
  (0, 30008)	0.12044278393722233
  (0, 32686)	0.19583745738826608
  (0, 41030)	0.18740770597476783
  (0, 41842)	0.21110333456364072
  (0, 33202)	0.20836071316587587
  (0, 34655)	0.18527679985191514
  (0, 42989)	0.22629114362711517
  (0, 10500)	0.22972949168440762
  (0, 8210)	0.15992547972496238
  (0, 43948)	0.17814847807432735
  (0, 41069)	0.29074072172372506
  (0, 31580)	0.23557272850227207
  (0, 4830)	0.19064477000743224
  (0, 29336)	0.15986094774869394
  (0, 47609)	0.3087146886722414
  (0, 45723)	0.17974911559001644


In [65]:
print(text_onehot[0])

  (0, 45723)	1
  (0, 47609)	1
  (0, 29336)	1
  (0, 4830)	1
  (0, 31580)	1
  (0, 41069)	1
  (0, 43948)	1
  (0, 8210)	1
  (0, 10500)	1
  (0, 42989)	1
  (0, 34655)	1
  (0, 33202)	1
  (0, 41842)	1
  (0, 41030)	1
  (0, 32686)	1
  (0, 30008)	1
  (0, 28161)	1
  (0, 45350)	1
  (0, 19528)	1
  (0, 11610)	1
  (0, 6283)	1


## **Analisis de Sentimientos mediante Machine Learning**

Convertimos las etiquetas a una representacion en entero (0,1,2)

In [39]:
y = []
for i in range(len(data_cleaned)):
    y.append(data_cleaned[i][0])
label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y)

Realizamos mediante sklearn la division de k-fold cross validation para tener 80% train y 20% test con un k de 5

In [38]:
from sklearn.linear_model import LogisticRegression
clf = SVC(kernel='linear')
logistic_reg = LogisticRegression(max_iter=1000)
decision_tree = DecisionTreeClassifier()

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


Finalmente, entrenamos cada uno de los modelos. Siendo en orden regresion logistica, SVM y arboles de desicion

In [42]:
# Realizar k-fold cross-validation y obtener los resultados de precisión
scores = cross_val_score(clf, text_tfidf, y, cv=cv)

# Imprimir los resultados de la cross-validation
print("Accuracy for each fold:", scores)
print("Mean accuracy:", scores.mean())

Accuracy for each fold: [0.72995833 0.726375   0.73283333 0.723375   0.72758333]
Mean accuracy: 0.7280249999999999


In [40]:
scores2 = cross_val_score(logistic_reg, text_tfidf, y, cv=cv)

print("Accuracy for each fold:", scores2)
print("Mean accuracy:", scores2.mean())

Accuracy for each fold: [0.72804167 0.72645833 0.73158333 0.72145833 0.72508333]
Mean accuracy: 0.726525


In [41]:
scores3 = cross_val_score(decision_tree, text_tfidf, y, cv=cv)

print("Accuracy for each fold:", scores3)
print("Mean accuracy:", scores3.mean())

Accuracy for each fold: [0.651625   0.64758333 0.6505     0.65041667 0.64595833]
Mean accuracy: 0.6492166666666666


## **Analisis de Sentimientos mediante diccionarios**

Llamamos a la funcion de ponderacion de polaridad de cada diccionario, guardando para cada reseña su prediccion con cada diccionario

In [78]:
text_analysis = []
hiv4 = ps.HIV4()

for i in range(len(text)):
    if i % 12000 == 0:
        print("Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)")
    harvard_pol_text = hiv4_polarity(text[i],hiv4)
    nltk_pol_text = analyze_sentiment(text[i])
    text_analysis.append([harvard_pol_text,nltk_pol_text])


Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)
Bloque aun siendo leido (de un total de 10 bloques de los 120k datos)


Observamos las primeras 10 predicciones por cada diccionario, guardamos en un csv cada prediccion (para no cargar la prediccion una y otra vez) y finalmente lo cargamos

In [79]:
print(text_analysis[:10])

[['Negativo', 'Positivo'], ['Positivo', 'Positivo'], ['Neutral', 'Positivo'], ['Neutral', 'Positivo'], ['Positivo', 'Positivo'], ['Neutral', 'Positivo'], ['Positivo', 'Positivo'], ['Neutral', 'Negativo'], ['Neutral', 'Positivo'], ['Neutral', 'Positivo']]


In [80]:
filename = "text_analysis.csv"

with open(filename, mode='w', newline='', encoding='utf-8') as archivo_csv:
    writing = csv.writer(archivo_csv)

    for fila in text_analysis:
        writing.writerow(fila)

print(f"Se ha guardado como {filename} correctamente.")

Se ha guardado como text_analysis.csv correctamente.


In [81]:
directory = os.getcwd()
path = (directory+"\\text_analysis.csv")
txt_analysis = leer_datos_noheader(path)

In [82]:
print(len(txt_analysis))

120000


Ponderamos la presicion al evaluarlo con la etiqueta real

In [83]:
accuracy_dict = [0,0]
for i in range(len(txt_analysis)):
    if data_cleaned[i][0] == txt_analysis[i][0]:
        accuracy_dict[0] += 1
    if data_cleaned[i][0] == txt_analysis[i][1]:
        accuracy_dict[1] += 1
        
accuracy_dict[0] = accuracy_dict[0] / 120000
accuracy_dict[1] = accuracy_dict[1] / 120000

In [87]:
print("Presicion usando diccionarios")
print("HIV4 ------------------ Opinion Lexicon")
print(accuracy_dict)

Presicion usando diccionarios
HIV4 ------------------ Opinion Lexicon
[0.3845166666666667, 0.4150333333333333]


## **Embeddings**

### Con capa de embeddings preconstruida

Preparamos la etiqueta y los datos para el embedding, padded_docs nos asegura que los documentos tengan un mismo tamaño

In [47]:
num_classes = 3

from keras.utils import to_categorical
y_cat = to_categorical(y, num_classes)



t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1

encoded_docs = t.texts_to_sequences(text)

# Definimos el maximo size de cada documento
max_length = 150
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')


Cargamos GloVe

In [48]:
embeddings_index = dict()
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
 values = line.split()
 word = values[0]
 coefs = asarray(values[1:], dtype='float32')
 embeddings_index[word] = coefs
f.close()

print('Loaded %s word vectors.' % len(embeddings_index))

embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
 embedding_vector = embeddings_index.get(word)
 if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector


Loaded 400000 word vectors.


Creamos los splits para kfold, ademas de que creamos el modelo de la red neuronal usando el embedding de glove ya entrenado, aplicando los pesos de la matriz de embeddings

In [49]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acc_scores = []

iteracion = 0
# Iteracion del modelo por KFolds
for train_index, test_index in kf.split(padded_docs, y):
    iteracion += 1
    print("[K-Fold] Iteracion: ",iteracion)

    model = Sequential()
    e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=150, trainable=False)
    model.add(e)
    model.add(Flatten())
    model.add(Dense(3, activation='softmax')) 
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    X_train, X_test = padded_docs[train_index], padded_docs[test_index]
    y_train, y_test = y_cat[train_index], y_cat[test_index]


    model.fit(X_train, y_train, epochs=50, verbose=0)


    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    acc_scores.append(accuracy)
    print(accuracy)

print("Precision promedio: {:.2f}%".format(np.mean(acc_scores) * 100))

[K-Fold] Iteracion:  1




0.5978749990463257
[K-Fold] Iteracion:  2
0.5911666750907898
[K-Fold] Iteracion:  3
0.5985000133514404
[K-Fold] Iteracion:  4
0.5980833172798157
[K-Fold] Iteracion:  5
0.5957083106040955
Precision promedio: 59.63%


### Con capa de embeddings aprendida mediante el cuerpo de textos

Realizamos el proceso de procesar nuestros documentos del corpus

In [52]:
vocab_size = 50000
# integer encode the documents
encoded_docs = [one_hot(d, vocab_size) for d in text]
#print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

En lugar de colocar la capa de embeddings precosntruida con GloVe, se aprendera mediante el cuerpo de textos, en este caso la dimensionalidad es menor aunque seguimos tomando el tamaño del vocabulario

In [53]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acc_scores = []

iteracion = 0

# Iteración del modelo por KFolds
for train_index, test_index in kf.split(padded_docs, y):
    iteracion += 1
    print("[K-Fold] Iteracion: ", iteracion)

    model = Sequential()

    model.add(Embedding(vocab_size, 8, input_length=max_length))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    X_train, X_test = padded_docs[train_index], padded_docs[test_index]
    y_train, y_test = y_cat[train_index], y_cat[test_index]

    model.fit(X_train, y_train, epochs=50, verbose=0)
    
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    acc_scores.append(accuracy)
    print(accuracy)

print("Precisión promedio: {:.2f}%".format(np.mean(acc_scores) * 100)) 

[K-Fold] Iteracion:  1
0.6744583249092102
[K-Fold] Iteracion:  2
0.6714583039283752
[K-Fold] Iteracion:  3
0.6805833578109741
[K-Fold] Iteracion:  4
0.6728749871253967
[K-Fold] Iteracion:  5
0.6694999933242798
Precisión promedio: 67.38%
